## 1. Check GPU Availability

In [1]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
DEVICE = None
if torch.cuda.is_available():
    DEVICE = "cuda"
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    DEVICE = "cpu"
    print("⚠️ No GPU detected! Go to Runtime → Change runtime type → GPU")

PyTorch version: 2.9.1+cu128
CUDA available: True
GPU: NVIDIA GeForce RTX 3050 Laptop GPU
GPU Memory: 4.3 GB


## 2. Mount Google Drive (for persistent storage)

In [ ]:
import sys
import os

# Mount Google Drive for persistent storage
from google.colab import drive
drive.mount('/content/drive')

# Create project directory in Drive
BASE_DIR = "/content/drive/MyDrive/ML_Sentiment_Analysis"
os.makedirs(os.path.join(BASE_DIR, 'checkpoints'), exist_ok=True)
os.makedirs(os.path.join(BASE_DIR, 'results'), exist_ok=True)
os.makedirs(os.path.join(BASE_DIR, 'plots'), exist_ok=True)

print("✅ Google Drive mounted!")
print(f"📁 Base directory: {BASE_DIR}")

IN_COLAB: True
📁 Using /content for storage
⚠️ Remember to download results before session ends!
BASE_DIR: /content


## 3. Clone/Upload Project

In [ ]:
import os

# ==================== CONFIGURE YOUR REPO HERE ====================
GITHUB_REPO = "https://github.com/R3dP4ndaXD/sentiment_analysis.git"
# ==================================================================

repo_name = "sentiment_analysis"
target_dir = f'/content/{repo_name}'

# Ensure we are in a stable directory before attempting to clone
# This helps avoid issues if the previous working directory was deleted
os.chdir('/content')

# Remove existing directory if it exists
if os.path.exists(target_dir):
    !rm -rf {target_dir}

# Clone from GitHub
!git clone {GITHUB_REPO} {target_dir}

# Change to project directory
os.chdir(target_dir)
print(f"✅ Cloned {GITHUB_REPO}")
print(f"📁 Working directory: {os.getcwd()}")

📥 Cloning from https://github.com/R3dP4ndaXD/sentiment_analysis...
✅ Downloaded and extracted to: /content/sentiment_analysis

📁 Working directory: /content/sentiment_analysis


In [2]:
# Verify project structure
!ls -la
print("\n📂 Source directory:")
!ls -la src/

total 1172
drwxr-xr-x  2 r3dp4nda r3dp4nda    4096 Jan  6 12:36 .
drwxr-xr-x 11 r3dp4nda r3dp4nda    4096 Jan  7 15:20 ..
-rw-r--r--  1 r3dp4nda r3dp4nda   21150 Jan 10 12:30 colab_experiment.ipynb
-rw-r--r--  1 r3dp4nda r3dp4nda 1150657 Jan  6 10:05 data_exploration.ipynb
-rw-r--r--  1 r3dp4nda r3dp4nda   16172 Jan 10 11:35 offline_augmentation_colab.ipynb

📂 Source directory:
ls: cannot access 'src/': No such file or directory


## 4. Install Dependencies

In [5]:
# Install from requirements.txt if exists
!pip install -q -r requirements.txt 2>/dev/null || echo "No requirements.txt found"

# Install core dependencies
!pip install -q torch pandas scikit-learn matplotlib seaborn spacy

# Download Romanian spaCy model
!python -m spacy download ro_core_news_sm -q

print("✅ Dependencies installed!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 127.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('ro_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
✅ Dependencies installed!


## 5. Download Dataset

In [10]:
import os
import pandas as pd

# Create data directories
os.makedirs('data/raw', exist_ok=True)
os.makedirs('data/processed', exist_ok=True)

# Download ro_sent dataset
TRAIN_URL = "https://raw.githubusercontent.com/dumitrescustefan/Romanian-Transformers/examples/examples/sentiment_analysis/ro/train.csv"
TEST_URL = "https://raw.githubusercontent.com/dumitrescustefan/Romanian-Transformers/examples/examples/sentiment_analysis/ro/test.csv"

!wget -q -O data/raw/train.csv "{TRAIN_URL}" 2>/dev/null || echo "Downloading train.csv..."
!wget -q -O data/raw/test.csv "{TEST_URL}" 2>/dev/null || echo "Downloading test.csv..."

# Check if download succeeded, if not try alternative method
if not os.path.exists('data/raw/train.csv') or os.path.getsize('data/raw/train.csv') < 1000:
    print("Trying alternative download method...")
    # Use datasets library as fallback
    !pip install -q datasets
    from datasets import load_dataset
    dataset = load_dataset("dumitrescustefan/ro_sent")
    dataset['train'].to_pandas().to_csv('data/raw/train.csv', index=False)
    dataset['test'].to_pandas().to_csv('data/raw/test.csv', index=False)

# Verify download
train_df = pd.read_csv('data/raw/train.csv')
test_df = pd.read_csv('data/raw/test.csv')
print(f"✅ Train samples: {len(train_df)}")
print(f"✅ Test samples: {len(test_df)}")
print(f"\nColumns: {train_df.columns.tolist()}")
print(f"\nLabel distribution (train):")
print(train_df['label'].value_counts())

✅ Train samples: 17941
✅ Test samples: 11005

Columns: ['index', 'text', 'label']

Label distribution (train):
label
1    11094
0     6847
Name: count, dtype: int64


In [ ]:
# Create train/val/test split
from sklearn.model_selection import train_test_split

# Split train into train/val (85/15)
train_data, val_data = train_test_split(
    train_df, 
    test_size=0.15, 
    random_state=42, 
    stratify=train_df['label']
)

# Save processed splits
DATA_DIR = "data/processed"
train_data.to_csv('data/processed/train.csv', index=False)
val_data.to_csv('data/processed/val.csv', index=False)
test_df.to_csv('data/processed/test.csv', index=False)

print(f"✅ Train: {len(train_data)} | Val: {len(val_data)} | Test: {len(test_df)}")

✅ Train: 16146 | Val: 1795 | Test: 11005


## 6. Run Experiments

Configure and run your training experiments below.

In [ ]:
# Paths for persistent storage on Google Drive
CHECKPOINT_DIR = "/content/drive/MyDrive/ML_Sentiment_Analysis/checkpoints"
RESULTS_DIR = "/content/drive/MyDrive/ML_Sentiment_Analysis/results"
PLOTS_DIR = "/content/drive/MyDrive/ML_Sentiment_Analysis/plots"

# Create directories if they don't exist
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR, exist_ok=True)

print(f"Checkpoint dir: {CHECKPOINT_DIR}")
print(f"Results dir: {RESULTS_DIR}")
print(f"Plots dir: {PLOTS_DIR}")

# ==================== EMBEDDINGS CONFIG ====================
# Download Romanian fastText embeddings (run once - ~4.5GB)
# !wget -q https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ro.300.bin.gz
# !gunzip cc.ro.300.bin.gz
# !mv cc.ro.300.bin /content/drive/MyDrive/ML_Sentiment_Analysis/

FASTTEXT_PATH = "/content/drive/MyDrive/ML_Sentiment_Analysis/cc.ro.300.bin"
FREEZE_EMBEDDINGS = False
# ============================================================


import os
import sys
from IPython import get_ipython

# Paths for persistent storage on Google Drive
CHECKPOINT_DIR = "/content/drive/MyDrive/ML_Sentiment_Analysis/checkpoints"
RESULTS_DIR = "/content/drive/MyDrive/ML_Sentiment_Analysis/results"
PLOTS_DIR = "/content/drive/MyDrive/ML_Sentiment_Analysis/plots"

# Create directories if they don't exist
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR, exist_ok=True)

print(f"Checkpoint dir: {CHECKPOINT_DIR}")
print(f"Results dir: {RESULTS_DIR}")
print(f"Plots dir: {PLOTS_DIR}")

# ==================== EMBEDDINGS CONFIG ====================
# Download Romanian fastText embeddings (run once - ~4.5GB)
# !wget -q https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ro.300.bin.gz
# !gunzip cc.ro.300.bin.gz
# !mv cc.ro.300.bin /content/drive/MyDrive/ML_Sentiment_Analysis/

FASTTEXT_PATH = "/content/drive/MyDrive/ML_Sentiment_Analysis/cc.ro.300.bin"
FREEZE_EMBEDDINGS = False
# ============================================================

# ==================== CONFIG HELPER ====================
def run_experiment(base_params, specific_params):
    """Helper to run experiments with specific configurations."""
    config = base_params.copy()
    config.update(specific_params)
    
    # Auto-generate experiment name if not set
    if 'experiment_name' not in config:
        aug_type = config.get('augment', 'none')
        aug_suffix = "_noaug" if aug_type == 'none' else f"_{aug_type}"
        if aug_type != 'none':
            aug_prob = config.get('aug_prob', 0.5)
            aug_suffix += f"_p{aug_prob}"

        model = config['model']
        if 'bi' not in config.get('model') and config.get('bidirectional'):
            model += '_bi'
        
        expand_factor = config.get('expand_factor', 1.0)
        expand_suffix = "" if expand_factor == 1.0 else f"_exp{expand_factor}"
        
        hidden_dim = config.get('hidden_dim', 256)
        hidden_dim_suffix = "" if hidden_dim == 256 else f"_hd{hidden_dim}"

        num_layers = config.get('num_layers', 2)
        num_layers_suffix = "" if num_layers == 2 else f"_nl{num_layers}"

        config['experiment_name'] = (
            f"{model}"
            f"{hidden_dim_suffix}"
            f"{expand_suffix}"
            f"{num_layers_suffix}"
            f"{aug_suffix}"
        )

    print(f"🚀 Starting Experiment: {config['experiment_name']}")
    
    # Build command string
    cmd_parts = ["python -m src.run_experiment"]
    for k, v in config.items():
        if isinstance(v, bool):
            if v: cmd_parts.append(f"--{k}")
        else:
            cmd_parts.append(f"--{k} {v}")
            
    cmd = " ".join(cmd_parts)
    print(f"Executing: {cmd}\n")
    
    # Execute using IPython magic for real-time output
    get_ipython().system(cmd)

# Base Configuration Map
BASE_CONFIG = {
    'model': 'lstm',
    'embedding_dim': 300,
    'hidden_dim': 128,
    'num_layers': 1,
    'dropout': 0.5,
    'pooling': 'max',
    'epochs': 30,
    'batch_size': 64,
    'lr': 0.0005,
    'weight_decay': 1e-5,
    'optimizer': 'adamw',
    'scheduler': 'plateau',
    'gradient_clip': 1.0,
    'max_seq_len': 160,
    'min_freq': 2,
    'max_vocab_size': 50000,
    'pretrained_embeddings': FASTTEXT_PATH,
    'early_stopping': 5,
    'checkpoint_metric': 'val_f1',
    'device': DEVICE,
    'data_dir': DATA_DIR,
    'checkpoint_dir': CHECKPOINT_DIR,
    'plots_dir': PLOTS_DIR,
    'results_dir': RESULTS_DIR,
    'remove_stopwords': False,
    'weighted_sampler': True,
    'balance_classes': False,
    'expand_factor': 1.0,
    'bidirectional': True,
    'num_workers': 2,
    'freeze_embeddings': True
}
    #--resume {PATH}
    #--evaluate_only /
    #--checkpoint {PATH} \
    #--verbose \
    #--no_plots \

Checkpoint dir: /content/checkpoints
Results dir: /content/results
Plots dir: /content/plots


No aug


In [ ]:
# No Augmentation Experiment
no_aug_params = {
    'augment': 'none',
    'num_workers': 2,
}

run_experiment(BASE_CONFIG, no_aug_params)

Aug



In [ ]:
# Augmentation Experiment
BALANCED_DATA_DIR = '/content/drive/MyDrive/ML_Sentiment_Analysis/data/balanced/'

aug_params = {
    'augment': 'eda_plus',
    'aug_prob': 0.1,
    'aug_mode': 'one_of',
    'data_dir': BALANCED_DATA_DIR,
    'num_workers': 0  # Lower workers for complex augmentation
}

## 7. View Results

In [ ]:
import json
import pandas as pd
from pathlib import Path

# Load all experiment summaries
results_dir = Path(RESULTS_DIR)
summaries = []

if results_dir.exists():
    for exp_dir in results_dir.iterdir():
        if exp_dir.is_dir():
            summary_file = exp_dir / 'summary.json'
            if summary_file.exists():
                with open(summary_file) as f:
                    summary = json.load(f)
                    summary['experiment'] = exp_dir.name
                    summaries.append(summary)

if summaries:
    df = pd.DataFrame(summaries)
    print("\n" + "="*60)
    print("EXPERIMENT RESULTS SUMMARY")
    print("="*60)
    display(df[['experiment', 'model', 'best_val_f1', 'best_val_acc', 'epochs_trained']].sort_values('best_val_f1', ascending=False))
else:
    print("No results found yet. Run experiments first!")

In [ ]:
# Display plots from experiments
from IPython.display import Image, display
from pathlib import Path

plots_dir = Path(PLOTS_DIR)
if plots_dir.exists():
    for exp_dir in sorted(plots_dir.iterdir()):
        if exp_dir.is_dir():
            print(f"\n📊 {exp_dir.name}")
            print("-" * 40)
            for plot in sorted(exp_dir.glob('*.png')):
                print(f"\n{plot.name}:")
                display(Image(filename=str(plot), width=600))
else:
    print(f"Plots directory not found: {PLOTS_DIR}")

## 8. Download Results to Local Machine

In [ ]:
# Zip and download all results
!zip -r /content/experiment_results.zip {RESULTS_DIR} {PLOTS_DIR}

from google.colab import files
files.download('/content/experiment_results.zip')
print("✅ Results downloaded!")

# Note: Results are also saved to Google Drive at:
print(f"\n📁 Results persist in Google Drive:")
print(f"   {RESULTS_DIR}")
print(f"   {PLOTS_DIR}")